# Load Packages

## For running method

In [1]:
import numpy as np
from scipy.linalg import eigh, block_diag, pinv
from scipy.linalg import eig
import torch
import pymanopt
from pymanopt.manifolds import Product, Stiefel
from pymanopt import Problem
from pymanopt.optimizers import ConjugateGradient
import autograd
from scipy.linalg import qr
from numpy.linalg import qr

## For loading data

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist, squareform

## For Evaluation

In [3]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist, squareform
from scipy.linalg import eigh, block_diag, pinv
import torch
import pymanopt
from pymanopt.manifolds import Product, Stiefel
from pymanopt import Problem
from pymanopt.optimizers import ConjugateGradient
import autograd
from scipy.stats import ortho_group
import matplotlib.pyplot as plt
import seaborn as sns
import os
from joblib import Parallel, delayed
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LogisticRegression
import matplotlib.colors as mcolors
from sklearn.metrics import accuracy_score, f1_score

# Functions

## Methods

In [4]:
def method1(X, Y1, Y2, d1, a, l):
    """
    Compute the desired matrices and eigenvectors as described in the prompt.

    Parameters:
        X (np.ndarray): Data matrix (n x p)
        Y1 (np.ndarray): Binary response vector of length n
        Y2 (np.ndarray): Binary response vector of length n
        d1 (int): Number of eigenvalues/vectors for the first block
        d2 (int): Number of eigenvalues/vectors for the second block
        a (float): hyperparameter for contrastive term
        l (float): hyperparameter for PCA term
    Returns:
        tuple: V1, V2 matrices corresponding to the extracted eigenvectors
    """
    # Total number of eigenvalues/vectors to extract
    d = d1
    # Check dimensions
    n, p = X.shape
    assert len(Y1) == n and len(Y2) == n, "Response vectors must have length n"
    assert d < p, "d must be less than the number of features p"
    #assert a >= 0, "contrastive hyperparameter must be nonnegative"
    #assert l >= 0, "PCA hyperparameter must be nonnegative"
    
    # Compute sample covariance of X
    Sigma = np.cov(X, rowvar=False)
    
    epsilon = 1e-5  # Small positive value
    Sigma_inv = np.linalg.inv(Sigma + epsilon * np.eye(Sigma.shape[0]))

    def compute_sigma_1(X, Y):
        """Compute Sigma_1 (or Sigma_2) as described."""
        p_1 = np.mean(Y)  # Proportion of slice Y = 1
        m_1 = np.mean(X[Y == 1], axis=0)  # Mean for Y = 1
        m_0 = np.mean(X[Y == 0], axis=0)  # Mean for Y = 0
        
        # Compute Sigma_1
        Sigma_1 = p_1 * np.outer(m_1, m_1) + (1 - p_1) * np.outer(m_0, m_0)
        return Sigma_1

    # Compute Sigma_1, Sigma_2, and Sigma_12
    Sigma_1 = compute_sigma_1(X, Y1)
    Sigma_2 = compute_sigma_1(X, Y2)
    
    # Compute A_1, A_2, A_12
    A_1 = Sigma @ Sigma_1 @ Sigma
    A_2 = Sigma @ Sigma_2 @ Sigma
    
    # Compute matrix to find top eigenvectors of
    M1 = Sigma_inv @ (a * A_1 - A_2) @ Sigma_inv + l * Sigma_inv
    M2 = Sigma_inv @ Sigma_inv
    
    M1 = (M1 + M1.T)/2
    M2 = (M2 + M2.T)/2
    
    #print("Symmetry check for M1:", np.allclose(M1, M1.T))
    #print("Symmetry check for M2:", np.allclose(M2, M2.T))

    
    #M2_cond_number = np.linalg.cond(M2)
    #print("M2 condition number = ", end="")
    #print(M2_cond_number)
    
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = eig(M1, M2)
    
    # Coerce eigenvalues and eigenvectors to real values
    eigenvalues = np.real(eigenvalues)
    eigenvectors = np.real(eigenvectors)
    
    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]
    
    #print("all eigenvalues = ", end="")
    #print(eigenvalues)
    
    # Select the top eigenvectors (e.g., top d1 eigenvectors)
    top_eigenvalues = eigenvalues[:d1]
    top_eigenvectors = eigenvectors[:, :d1]
    
    #print("top eigenvalues = ", end="")
    #print(top_eigenvalues)
    
    #print("pct var = ", end="")
    #print(sum(top_eigenvalues) / sum(eigenvalues) * 100)
    
    # Rotate back
    V = Sigma_inv @ top_eigenvectors
    
    # Perform QR decomposition to orthogonalize V
    Q, R = np.linalg.qr(V)
    
    return np.real(Q)

In [5]:
def method2(X, Y1, Y2, d1, d2, e, a, b, c, l):
    """
    Compute the desired matrices and eigenvectors as described in the prompt.

    Parameters:
        X (np.ndarray): Data matrix (n x p)
        Y1 (np.ndarray): Binary response vector of length n
        Y2 (np.ndarray): Binary response vector of length n
        d1 (int): Number of eigenvalues/vectors for the first block
        d2 (int): Number of eigenvalues/vectors for the second block
        a (float): hyperparameter for contrastive term (alpha)
        b (float): hyperparameter for V2 (beta)
        c (float): hyperparameter for V2 contrastive term (gamma)
        l (float): hyperparameter for PCA term (lambda)
    Returns:
        tuple: V1, V2 matrices corresponding to the extracted eigenvectors
    """
    # Total number of eigenvalues/vectors to extract
    d = d1 + d2
    # Check dimensions
    n, p = X.shape
    assert len(Y1) == n and len(Y2) == n, "Response vectors must have length n"
    assert d < p, "d must be less than the number of features p"
    assert a >= 0, "contrastive hyperparameter must be nonnegative"
    assert b >= 0, "hyperparameter for V2 must be nonnegative"
    assert c >= 0, "hyperparameter for V2 contrastive term must be nonnegative"
    assert l >= 0, "PCA hyperparameter must be nonnegative"
    assert e >= 0, "first hyperparameter must be nonnegative"
    assert max(a, b, c, l, e) >= 0, "at least one hyperparameter must be positive"
    
    # Compute sample covariance of X
    Sigma = np.cov(X, rowvar=False)
    #Sigma_inv = np.linalg.inv(Sigma)
    B = Sigma @ Sigma

    def compute_sigma_1(X, Y):
        """Compute Sigma_1 (or Sigma_2) as described."""
        p_1 = np.mean(Y)  # Proportion of slice Y = 1
        m_1 = np.mean(X[Y == 1], axis=0)  # Mean for Y = 1
        m_0 = np.mean(X[Y == 0], axis=0)  # Mean for Y = 0
        
        # Compute Sigma_1
        Sigma_1 = p_1 * np.outer(m_1, m_1) + (1 - p_1) * np.outer(m_0, m_0)
        return Sigma_1

    # Compute Sigma_1, Sigma_2, and Sigma_12
    Sigma_1 = compute_sigma_1(X, Y1)
    Sigma_2 = compute_sigma_1(X, Y2)
    
    # Compute A_1, A_2, A_12
    A_1 = Sigma @ Sigma_1 @ Sigma
    A_2 = Sigma @ Sigma_2 @ Sigma
    
    # V1
    # Compute matrix to find top eigenvectors of
    M1 = Sigma @ (A_1 - a * A_2) @ Sigma
    
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors1 = np.linalg.eig(M1)
    
    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors1 = eigenvectors1[:, sorted_indices]
    
    # Select the top eigenvectors (e.g., top d1 eigenvectors)
    top_eigenvectors1 = eigenvectors1[:, :d1]
    
    # Rotate back
    V1 = np.linalg.solve(Sigma, top_eigenvectors1)
    
    # Perform QR decomposition to orthogonalize V
    Q1, R = np.linalg.qr(V1)
    
    
    # V2
    # Compute matrix to find top eigenvectors of
    M2 = Sigma @ (c * A_2 - b * A_1) @ Sigma
    
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors2 = np.linalg.eig(M2)
    
    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors2 = eigenvectors2[:, sorted_indices]
    
    # Select the top eigenvectors (e.g., top d1 eigenvectors)
    top_eigenvectors2 = eigenvectors2[:, :d1]
    
    # Rotate back
    V2 = np.linalg.solve(Sigma, top_eigenvectors2)
    
    # Perform QR decomposition to orthogonalize V
    Q2, R = np.linalg.qr(V2)
    
    if l == 0:
        return np.real(Q1), np.real(Q2)
    
    else:
        
        # Convert necessary matrices to PyTorch tensors
        Sigma_cubed = torch.tensor(Sigma @ B, dtype=torch.float64)
        A_1_torch = torch.tensor(A_1, dtype=torch.float64)
        A_2_torch = torch.tensor(A_2, dtype=torch.float64)
        B_torch = torch.tensor(B, dtype=torch.float64)
        
        # Define the product manifold
        manifold = pymanopt.manifolds.Product([
            pymanopt.manifolds.Stiefel(p, d1),
            pymanopt.manifolds.Stiefel(p, d2)
        ])

        
        @pymanopt.function.pytorch(manifold)
        def cost(V_1, V_2):
            V_combined = torch.cat([V_1, V_2], dim=1)
            part1 = -torch.trace(V_1.T @ A_1_torch @ V_1)
            part2 = -torch.trace(V_2.T @ A_2_torch @ V_2)
            VDV = V_combined.T @ B_torch @ V_combined
            VDV_inv = torch.linalg.pinv(VDV)
            part3 = -l * torch.trace(V_combined.T @ Sigma_cubed @ V_combined @ VDV_inv)
            return part1 + part2 + part3

        # Create the optimization problem
        problem = Problem(manifold=manifold, cost=cost)

        # Use a conjugate gradient optimizer
        optimizer = ConjugateGradient()
        result = optimizer.run(problem)

        # Extract optimized V_1 and V_2
        V_1_opt, V_2_opt = result.point
        
        # Rotate back
        V1 = np.linalg.solve(Sigma, V_1_opt)
        V2 = np.linalg.solve(Sigma, V_2_opt)
        
        # Perform QR decomposition to orthogonalize V
        Q1, R = np.linalg.qr(V1)
        Q2, R = np.linalg.qr(V2)
        
        return Q1, Q2

In [6]:
def fairPCA(X, Y, d, m):
    assert(m >= 1)
    
    # Compute the covariance matrix
    Sigma = np.cov(X, rowvar=False)
    
    # Split the data into two groups based on Y
    X0 = X[Y == 0]
    X1 = X[Y == 1]
    
    # Compute mean vectors
    mu0 = np.mean(X0, axis=0)
    mu1 = np.mean(X1, axis=0)
    
    # Compute covariance matrices
    Sigma0 = np.cov(X0, rowvar=False)
    Sigma1 = np.cov(X1, rowvar=False)
    
    # Compute the second moment difference
    S = Sigma1 + np.outer(mu1, mu1) - Sigma0 - np.outer(mu0, mu0)
    
    # Compute eigenvalues and eigenvectors of Sigma_diff
    eigvals, eigvecs = np.linalg.eigh(S)
    
    # Sort eigenvectors by absolute eigenvalues in descending order
    sorted_indices = np.argsort(-np.abs(eigvals))
    top_m_eigvecs = eigvecs[:, sorted_indices[:m]]
    
    # Include mu0 - mu1 in the subspace
    #mu_diff = (mu0 - mu1).reshape(-1, 1)
    #combined_basis = np.hstack([top_m_eigvecs, mu_diff])
    
    # Perform QR decomposition to obtain an orthonormal basis
    Q, _ = np.linalg.qr(top_m_eigvecs, mode='reduced')
    I = np.eye(X.shape[1])  # Identity matrix of size p x p
    Pi = I - Q @ Q.T  # Project onto orthogonal complement
    
    # Compute Pi Sigma Pi
    transformed_Sigma = Pi @ Sigma @ Pi
    
    # Compute the top d eigenvectors of Pi Sigma Pi
    eigvals_final, eigvecs_final = np.linalg.eigh(transformed_Sigma)
    sorted_indices_final = np.argsort(-eigvals_final)
    top_d_eigvecs = eigvecs_final[:, sorted_indices_final[:d]]
    
    return top_d_eigvecs

In [7]:
def SIR(X, Y, d):
    # Compute sample covariance of X
    Sigma = np.cov(X, rowvar=False)
    
    epsilon = 1e-5  # Small positive value
    Sigma_inv = np.linalg.inv(Sigma + epsilon * np.eye(Sigma.shape[0]))

    def compute_sigma_1(X, Y):
        """Compute Sigma_1 (or Sigma_2) as described."""
        p_1 = np.mean(Y)  # Proportion of slice Y = 1
        m_1 = np.mean(X[Y == 1], axis=0)  # Mean for Y = 1
        m_0 = np.mean(X[Y == 0], axis=0)  # Mean for Y = 0
        
        # Compute Sigma_1
        Sigma_1 = p_1 * np.outer(m_1, m_1) + (1 - p_1) * np.outer(m_0, m_0)
        return Sigma_1

    # Compute Sigma_1, Sigma_2, and Sigma_12
    Sigma_1 = compute_sigma_1(X, Y)
    
    # Compute A_1, A_2, A_12
    A_1 = Sigma @ Sigma_1 @ Sigma
    
    # Compute matrix to find top eigenvectors of
    M = Sigma_inv @ A_1 @ Sigma_inv
    
    M = (M + M.T)/2

    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = eig(M)
    
    # Coerce eigenvalues and eigenvectors to real values
    eigenvalues = np.real(eigenvalues)
    eigenvectors = np.real(eigenvectors)
    
    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]
    
    # Select the top eigenvectors (e.g., top d1 eigenvectors)
    top_eigenvalues = eigenvalues[:d]
    top_eigenvectors = eigenvectors[:, :d]
    
    # Rotate back
    V = Sigma_inv @ top_eigenvectors
    
    # Perform QR decomposition to orthogonalize V
    Q, R = np.linalg.qr(V)
    
    return np.real(Q)

## Evaluation

### Maximum Mean Discrepancy

In [8]:
def mmd(X1, X2, sigma, batch_size=1000):
    """
    Compute the Maximum Mean Discrepancy (MMD) between two datasets X1 and X2 with memory efficiency.
    """
    def rbf_batch(X1, X2, sigma):
        """
        Compute the Radial Basis Function (RBF) kernel between two datasets in batches.
        """
        result = 0.0
        for i in range(0, X1.shape[0], batch_size):
            X1_batch = X1[i:i + batch_size]
            distances = cdist(X1_batch, X2, 'sqeuclidean')
            result += np.sum(np.exp(-distances / (2 * sigma**2)))
        return result

    m = X1.shape[0]
    n = X2.shape[0]

    # Term 1: Intra-set distances for X1
    term1 = (1 / m**2) * rbf_batch(X1, X1, sigma)

    # Term 2: Intra-set distances for X2
    term2 = (1 / n**2) * rbf_batch(X2, X2, sigma)

    # Term 3: Inter-set distances between X1 and X2
    term3 = (2 / (m * n)) * rbf_batch(X1, X2, sigma)

    d = term1 + term2 - term3
    return d

### Percent Variance

In [9]:
def pct_var(X, V):
    """
    Compute the % of variance retained by the projection in the reduced dataset
    """
    
    # Compute sample covariance of X
    Sigma = np.cov(X, rowvar=False)
    
    return 100 * np.trace(V.T @ Sigma @ V) / np.trace(Sigma) 

### Percent Accuracy (for Y1)

In [10]:
def compute_svm_metrics(X_reduced_data, Y1_labels, Y2_groups, kernel='rbf', C=1.0, gamma='scale', cv=5):
    """
    Compute cross-validated Accuracy (%Acc), F1 score (%F1), and Demographic Parity difference (%∆DP) for a kernel SVM.
    
    Parameters:
    - X_reduced_data: np.ndarray, the reduced data matrix (n_samples, n_features)
    - Y1_labels: np.ndarray, the target labels (n_samples,)
    - Y2_groups: np.ndarray, the demographic group labels (n_samples,)
    - kernel: str, the kernel type to use for SVM ('linear', 'rbf', etc.)
    - C: float, regularization parameter for SVM
    - gamma: str or float, kernel coefficient for 'rbf', 'poly', and 'sigmoid'
    - cv: int, the number of cross-validation folds

    Returns:
    - mean_acc, std_acc: Mean and standard deviation of accuracy (%Acc)
    - mean_f1, std_f1: Mean and standard deviation of F1 score (%F1)
    - mean_dp, std_dp: Mean and standard deviation of demographic parity difference (%∆DP)
    """
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    svm = SVC(kernel=kernel, C=C, gamma=gamma)
    
    acc_scores = []
    f1_scores = []
    dp_differences = []
    
    for train_index, test_index in skf.split(X_reduced_data, Y1_labels):
        X_train, X_test = X_reduced_data[train_index], X_reduced_data[test_index]
        y_train, y_test = Y1_labels[train_index], Y1_labels[test_index]
        g_test = Y2_groups[test_index]
        
        svm.fit(X_train, y_train)
        y_pred = svm.predict(X_test)
        
        # Compute Accuracy and F1 Score
        acc_scores.append(accuracy_score(y_test, y_pred) * 100)
        f1_scores.append(f1_score(y_test, y_pred) * 100)
        
        # Compute Demographic Parity difference
        positive_rates = {}
        for group in np.unique(g_test):
            group_indices = np.where(g_test == group)[0]
            positive_rate = np.mean(y_pred[group_indices] == 1)
            positive_rates[group] = positive_rate
        
        group_rates = list(positive_rates.values())
        if len(group_rates) == 2:  # Ensure binary groups
            dp_difference = abs(group_rates[0] - group_rates[1])
            dp_differences.append(dp_difference * 100)
        
    return (
        np.mean(acc_scores), np.std(acc_scores),
        np.mean(f1_scores), np.std(f1_scores),
        np.mean(dp_differences), np.std(dp_differences)
    )

### Demographic Parity (predictive power of Y1 between the Y2 groups)

In [11]:
def pct_delta_demographic_parity(X_reduced_data, Y1_labels, Y2_groups, kernel='rbf', C=1.0, gamma='scale', cv=5):
    """
    Compute %∆DP (Demographic Parity) for a kernel SVM.

    Parameters:
    - X_reduced_data: np.ndarray, the reduced data matrix (n_samples, n_features)
    - Y2_labels: np.ndarray, the target labels (n_samples,)
    - groups: np.ndarray, the demographic group labels (n_samples,)
    - kernel: str, the kernel type to use for SVM ('linear', 'rbf', etc.)
    - C: float, regularization parameter for SVM
    - gamma: str or float, kernel coefficient for 'rbf', 'poly', and 'sigmoid'
    - cv: int, the number of cross-validation folds

    Returns:
    - mean_dp: float, mean demographic parity (%∆DP) across folds
    """
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    svm = SVC(kernel=kernel, C=C, gamma=gamma)

    dp_differences = []

    for train_index, test_index in skf.split(X_reduced_data, Y1_labels):
        X_train, X_test = X_reduced_data[train_index], X_reduced_data[test_index]
        y_train, y_test = Y1_labels[train_index], Y1_labels[test_index]
        g_test = Y2_groups[test_index]

        svm.fit(X_train, y_train)
        y_pred = svm.predict(X_test)

        # Compute positive prediction rates per group
        positive_rates = {}
        for group in np.unique(g_test):
            group_indices = np.where(g_test == group)[0]
            positive_rate = np.mean(y_pred[group_indices] == 1)
            positive_rates[group] = positive_rate

        # Compute absolute differences between groups
        group_rates = list(positive_rates.values())
        dp_difference = abs(group_rates[0] - group_rates[1])  # Adjust for two-group assumption
        dp_differences.append(dp_difference)

    mean_dp = np.mean(dp_differences) * 100  # Convert to percentage
    sd_dp = np.std(dp_differences) * 100 # Convert to percentage
    return mean_dp, sd_dp

In [12]:
def pct_delta_demographic_parity_logistic(X_reduced_data, Y1_labels, Y2_groups, C=1.0, cv=5):
    """
    Compute %∆DP (Demographic Parity) for a Logistic Regression classifier.

    Parameters:
    - X_reduced_data: np.ndarray, the reduced data matrix (n_samples, n_features)
    - Y1_labels: np.ndarray, the target labels (n_samples,)
    - Y2_groups: np.ndarray, the demographic group labels (n_samples,)
    - C: float, regularization parameter for Logistic Regression (default 1.0)
    - cv: int, the number of cross-validation folds (default 5)

    Returns:
    - mean_dp: float, mean demographic parity (%∆DP) across folds
    - sd_dp: float, standard deviation of demographic parity across folds
    """
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    logistic_model = LogisticRegression(C=C, solver="liblinear", random_state=42)  # Use liblinear for small datasets

    dp_differences = []

    for train_index, test_index in skf.split(X_reduced_data, Y1_labels):
        X_train, X_test = X_reduced_data[train_index], X_reduced_data[test_index]
        y_train, y_test = Y1_labels[train_index], Y1_labels[test_index]
        g_test = Y2_groups[test_index]

        logistic_model.fit(X_train, y_train)
        y_pred = logistic_model.predict(X_test)

        # Compute positive prediction rates per group
        positive_rates = {}
        for group in np.unique(g_test):
            group_indices = np.where(g_test == group)[0]
            positive_rate = np.mean(y_pred[group_indices] == 1)
            positive_rates[group] = positive_rate

        # Compute absolute differences between groups
        group_rates = list(positive_rates.values())
        dp_difference = abs(group_rates[0] - group_rates[1])  # Adjust for two-group assumption
        dp_differences.append(dp_difference)

    mean_dp = np.mean(dp_differences) * 100  # Convert to percentage
    sd_dp = np.std(dp_differences) * 100  # Convert to percentage
    return mean_dp, sd_dp


### Visualization

In [13]:
def plot_reduced_X(X, V, W, method_name="DR", W_name="Group", group_level_0="Class 0", group_level_1="Class 1", caption=None):
    """
    Projects X onto the factor loading matrix V and plots the reduced data, 
    colored by the binary variable W.

    Parameters:
    - X (np.ndarray): The original data matrix of shape (n, p).
    - V (np.ndarray): The factor loading matrix of shape (p, 2).
    - W (np.ndarray): A binary variable of shape (n,) used for coloring.
    - method_name (str): Name of the dimension reduction method (e.g., "SIRFairRe").
    - W_name (str): Name of the binary variable for labeling.
    - group_level_0 (str): Label for group 0.
    - group_level_1 (str): Label for group 1.
    - caption (str, optional): Caption to display below the plot.

    Returns:
    - None (displays a scatter plot).
    """
    # Compute the reduced representation (n x 2)
    X_reduced = X @ V  # Matrix multiplication
    cmap = mcolors.ListedColormap(['red', 'blue'])  # 0 → Red, 1 → Blue

    # Create the scatter plot
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=W, cmap=cmap, edgecolors="k", alpha=0.7)

    # Create a custom legend
    handles = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=8, label=group_level_0),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=8, label=group_level_1)
    ]
    plt.legend(handles=handles, title=W_name, fontsize=16)

    # Label axes dynamically based on method name
    plt.xlabel(f"{method_name} 1", fontsize=20)
    plt.ylabel(f"{method_name} 2", fontsize=20)
    plt.title(f"{method_name} labeled by {W_name}", fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    # Add caption if provided
    if caption:
        plt.figtext(0.5, -0.05, caption, wrap=True, horizontalalignment='center', fontsize=20)

    # Save the plot before displaying
    filename = f"ADULT_Reduced_Dim_{method_name.replace('%', '').replace(' ', '_')}_{W_name.replace(' ', '_')}.png"
    plt.savefig(filename, dpi=300, bbox_inches="tight")

    # Show the plot
    plt.show()


# Load Datasets

## Adult UCI

In [14]:
# Load your dataset
UCI_adult = pd.read_csv('full.csv')

# Assuming the last two columns are sensitive attribute and target
X = UCI_adult.iloc[:, :-2].values
Y = UCI_adult.iloc[:, -2].values  # Target variable
Z = UCI_adult.iloc[:, -1].values  # Sensitive attribute

In [ ]:
np.unique(Y, return_counts = True)

In [ ]:
np.unique(Z, return_counts = True)

In [17]:
# Standardize only the continuous features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Assume X is your data matrix (n_samples x n_features)
# Standardize X before computing PCA
X_centered = X - np.mean(X, axis=0)  # Centering the data (zero mean)

# Compute the covariance matrix
cov_matrix = np.cov(X_centered, rowvar=False)

# Compute eigenvalues (variances along principal components)
eigenvalues, _ = np.linalg.eigh(cov_matrix)

# Sort eigenvalues in descending order
eigenvalues = np.sort(eigenvalues)[::-1]

# Compute the explained variance ratio
explained_variance_ratio = eigenvalues / np.sum(eigenvalues)

# Cumulative variance
cumulative_variance = np.cumsum(explained_variance_ratio)

# Plot Scree Plot
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(eigenvalues) + 1), explained_variance_ratio, marker='o', linestyle='--', label="Explained Variance")
plt.plot(range(1, len(eigenvalues) + 1), cumulative_variance, marker='s', linestyle='-', label="Cumulative Variance")

plt.xlabel("Number of Principal Components")
plt.ylabel("Variance Explained")
plt.title("Scree Plot")
plt.axhline(y=0.50, color='r', linestyle="--", label="50% Variance Threshold")  # Highlight 90% variance line
plt.legend()
plt.savefig("scree_plot_ADULT.png", dpi=300)
plt.grid()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to perform PCA and return eigenvalues
def compute_eigenvalues(X):
    X_centered = X - np.mean(X, axis=0)  # Centering the data
    cov_matrix = np.cov(X_centered, rowvar=False)
    eigenvalues, _ = np.linalg.eigh(cov_matrix)
    return np.sort(eigenvalues)[::-1]  # Sort in descending order

# Compute actual eigenvalues
eigenvalues_real = compute_eigenvalues(X)

# Permutation test parameters
num_permutations = 100  # Number of shuffled datasets
n_samples, n_features = X.shape
permuted_eigenvalues = np.zeros((num_permutations, n_features))

# Generate permuted datasets and compute eigenvalues
for i in range(num_permutations):
    X_permuted = np.copy(X)
    for j in range(n_features):
        np.random.shuffle(X_permuted[:, j])  # Shuffle each feature independently
    permuted_eigenvalues[i, :] = compute_eigenvalues(X_permuted)

# Compute 95th percentile of permuted eigenvalues (null distribution threshold)
thresholds = np.percentile(permuted_eigenvalues, 95, axis=0)

# Determine the number of dimensions d where real eigenvalues exceed the threshold
d = np.sum(eigenvalues_real > thresholds)

# Scree plot with threshold
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(eigenvalues_real) + 1), eigenvalues_real, marker='o', linestyle='--', label="Real Data")
plt.plot(range(1, len(thresholds) + 1), thresholds, marker='x', linestyle='-', label="95% Permutation Threshold")
plt.xlabel("Number of Principal Components")
plt.ylabel("Eigenvalue")
plt.title("Scree Plot with Permutation Test Threshold")
plt.legend()
plt.grid()

# Save the plot
plt.savefig("scree_plot_permutation.png", dpi=300)
plt.show()

print(f"Estimated dimension (d) based on permutation test: {d}")

In [ ]:
d

In [ ]:
X.shape

In [22]:
X0 = X[Z== 0]  # Rows where Z == 0
X1 = X[Z == 1]  # Rows where Z == 1

In [ ]:
UCI_adult.shape

### Preprocessing so that X0 and X1 are separately centered

In [24]:
# Split X based on S
X_0, X_1 = X[Z == 0], X[Z == 1]
    
# Compute group-wise means
mu_0, mu_1 = X_0.mean(axis=0), X_1.mean(axis=0)
    
# Center each group separately
X_0_centered = X_0 - mu_0
X_1_centered = X_1 - mu_1

# Reassemble into original order
X_centered = np.empty_like(X)
X_centered[Z == 0] = X_0_centered
X_centered[Z == 1] = X_1_centered

In [ ]:
# Assume X is your data matrix (n_samples x n_features)
# Standardize X before computing PCA
X_centered = X - np.mean(X, axis=0)  # Centering the data (zero mean)

# Compute the covariance matrix
cov_matrix = np.cov(X_centered, rowvar=False)

# Compute eigenvalues (variances along principal components)
eigenvalues, _ = np.linalg.eigh(cov_matrix)

# Sort eigenvalues in descending order
eigenvalues = np.sort(eigenvalues)[::-1]

# Compute the explained variance ratio
explained_variance_ratio = eigenvalues / np.sum(eigenvalues)

# Cumulative variance
cumulative_variance = np.cumsum(explained_variance_ratio)

# Plot Scree Plot
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(eigenvalues) + 1), explained_variance_ratio, marker='o', linestyle='--', label="Explained Variance")
plt.plot(range(1, len(eigenvalues) + 1), cumulative_variance, marker='s', linestyle='-', label="Cumulative Variance")

plt.xlabel("Number of Principal Components")
plt.ylabel("Variance Explained")
plt.title("Scree Plot")
plt.axhline(y=0.50, color='r', linestyle="--", label="50% Variance Threshold")  # Highlight 90% variance line
plt.legend()
plt.grid()
plt.show()


# Parameter Settings

In [26]:
x_values = [-0.5, -0.25, 0, 0.25, 0.5, 1]
y_values = [0, 0.5, 1, 5, 10]

parameters_list = [[x, y] for x in x_values for y in y_values]

In [ ]:
parameters_list

In [ ]:
parameters_list[3]

# Compute All Loadings of Interest

## Option 1

In [28]:
Vs = dict()

In [29]:
for setting in parameters_list:
    al = tuple(setting)
    a, l, = al
    V = method1(X, Y, Z, 2, a, l)
    Vs[al] = V

In [ ]:
len(Vs)

## PCA

In [31]:
Sigma = np.cov(X, rowvar=False)

In [32]:
eigenvalues, eigenvectors = eigh(Sigma)

In [33]:
V_PCA = eigenvectors[:, -2:]

## SIR(Y1)

In [34]:
V_SIRY1 = SIR(X, Y, 2)

## -SIR(Y2)

In [35]:
V_negSIRY2 = method1(X, Y, Z, 2, 0, 0)

## FairPCA

In [36]:
Vs_fair = dict()

In [37]:
ms = [1, 15, 50]

In [38]:
for m in ms:
    V = fairPCA(X, Z, 2, m)
    Vs_fair[m] = V

In [ ]:
len(ms)

# Perform Data Splits

In [40]:
def get_splits(X, Y1_labels, cv=5, random_state=42):
    """Precompute and store cross-validation splits."""
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=random_state)
    splits = [(train_idx, test_idx) for train_idx, test_idx in skf.split(X, Y1_labels)]
    return splits

In [41]:
def compute_sigma(X_reduced, sample_frac=0.1):
    """Compute sigma efficiently using a subset of the data."""
    n_samples = max(10, int(sample_frac * X_reduced.shape[0]))  # At least 10 samples
    X_sample = X_reduced[np.random.choice(X_reduced.shape[0], n_samples, replace=False)]
    pairwise_sq_distances = pdist(X_sample, 'sqeuclidean')
    return np.sqrt(np.median(pairwise_sq_distances) / 2)

In [42]:
# Define process_factor_loading globally and explicitly pass parameters
def process_factor_loading(V, i, X, Y1_labels, Y2_groups, sample_frac, kernel, C, gamma, cv, splits):
    """Compute all metrics for a given factor loading."""
    print(f"Processing factor loading {i}")
    
    X_reduced = X @ V  # Project data

    # Compute MMD sigma
    sigma = compute_sigma(X_reduced, sample_frac=sample_frac)

    # Compute metrics
    mmd_score = mmd(X_reduced[Y1_labels == 0], X_reduced[Y1_labels == 1], sigma)
    var_pct = pct_var(X, V)

    # Compute accuracy and demographic parity
    acc_mean, acc_std, f1_mean, f1_std, dp_mean, dp_std = compute_svm_metrics(X_reduced, Y1_labels, Y2_groups, kernel='rbf', C=1.0, gamma='scale', cv=5)
    #acc_mean, acc_std = pct_accuracy(X_reduced, Y1_labels, kernel=kernel, C=C, gamma=gamma, cv=cv)
    #dp_mean, dp_std = pct_delta_demographic_parity(X_reduced, Y1_labels, Y2_groups, kernel=kernel, C=C, gamma=gamma, cv=cv)
    #dp_mean_logistic, dp_std_logistic = pct_delta_demographic_parity_logistic(X_reduced, Y1_labels, Y2_groups, C=C, cv=cv)

    # Store results
    return {
        'Factor_Loading_Index': i,
        'MMD': mmd_score,
        '% Variance': var_pct,
        '% Accuracy Mean': acc_mean,
        '% Accuracy Std': acc_std,
        'F1 Mean': f1_mean,
        'F1 Std': f1_std,
        '% Delta DP Mean': dp_mean,
        '% Delta DP Std': dp_std
        #'% Delta DP Mean Logistic': dp_mean_logistic,
        #'% Delta DP Std Logistic': dp_std_logistic
    }

# Modify run_simulation to explicitly pass all arguments
def run_simulation(X, Y1_labels, Y2_groups, factor_loadings, cv=5, sample_frac=0.1, kernel='rbf', C=1.0, gamma='scale'):
    """
    Runs the simulation for all factor loadings and stores results in a DataFrame.
    """
    splits = get_splits(X, Y1_labels, cv=cv)  # Precompute cross-validation splits

    # Use joblib to parallelize across factor loadings
    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(process_factor_loading)(V, i, X, Y1_labels, Y2_groups, sample_frac, kernel, C, gamma, cv, splits) 
        for i, V in enumerate(factor_loadings)
    )

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
factor_loadings = list(Vs.values()) + list(Vs_fair.values())
factor_loadings.append(V_PCA)
factor_loadings.append(V_SIRY1)
factor_loadings.append(V_negSIRY2)

In [44]:
sample_frac = 0.1
C = 1
gamma = "scale"
kernel = "linear"
cv = 5
splits = get_splits(X, Y, cv=cv)

In [ ]:
from tqdm import tqdm
from joblib import Parallel, delayed

# Wrap `enumerate(factor_loadings)` with `list()` so `tqdm` can track progress
results = Parallel(n_jobs=-1, verbose=10)(
    delayed(process_factor_loading)(V, i, X, Y, Z, sample_frac, kernel, C, gamma, cv, splits) 
    for i, V in tqdm(list(enumerate(factor_loadings)), total=len(factor_loadings))
)

# Convert list of results to a DataFrame
results_df = pd.DataFrame(results)

# Save as a tab-separated text file
results_df.to_csv("simulation_results.txt", sep="\t", index=False)

In [14]:
# Load the results from the saved file
results_df = pd.read_csv("simulation_results.txt", sep="\t")

In [ ]:
import time

# Track start time
start_time = time.time()

# Train a linear SVM on the full dataset using parallel cross-validation
clf = SVC(kernel="linear", C=1, random_state=42)
accuracy_scores = cross_val_score(clf, X, Y, cv=5, n_jobs=-1)  # Enables parallel execution

# Compute mean and standard deviation of accuracy
accuracy_mean = accuracy_scores.mean()
accuracy_std = accuracy_scores.std()

# Track end time
end_time = time.time()

print(f"Prediction Accuracy on Full Dataset: {accuracy_mean:.4f} ± {accuracy_std:.4f}")
print(f"Total Runtime: {end_time - start_time:.2f} seconds")


In [15]:
accuracy_mean = 0.8465

# Highlighting Key Results from results_df

In [ ]:
import pandas as pd

# Define the index ranges
first_category = results_df.iloc[:30]

# Function to get required rows
def get_selected_rows(df):
    rows = []
    # 1. The one with minimum MMD
    rows.append(df.loc[df['MMD'].idxmin()])

    # 2. Of the ones with MMD < 0.01, the one with maximum % Variance
    subset = df[df['MMD'] < 0.05]
    if not subset.empty:
        rows.append(subset.loc[subset['% Variance'].idxmax()])

    # 3. Of the ones with MMD < 0.06, the one with maximum % Accuracy Mean
    subset = df[df['MMD'] < 0.1]
    if not subset.empty:
        rows.append(subset.loc[subset['% Accuracy Mean'].idxmax()])

    # 4. The one with maximum % Accuracy Mean
    rows.append(df.loc[df['% Accuracy Mean'].idxmax()])

    # 5. Of the ones with MMD < 0.06, the one with maximum F1 Mean
    subset = df[df['MMD'] < 0.1]
    if not subset.empty:
        rows.append(subset.loc[subset['F1 Mean'].idxmax()])

    # 6. Of the ones with % Delta DP Mean < 0.01, the one with maximum % Variance
    subset = df[df['% Delta DP Mean'] < 0.01]
    if not subset.empty:
        rows.append(subset.loc[subset['% Variance'].idxmax()])

    # 7. Of the ones with % Delta DP Mean < 1, the one with maximum % Variance
    subset = df[df['% Delta DP Mean'] < 1]
    if not subset.empty:
        rows.append(subset.loc[subset['% Variance'].idxmax()])

    return pd.DataFrame(rows)

# Get selected rows for each category
selected_first = get_selected_rows(first_category)

# Select only the rows with indices 30, 31, and 32
second_category = results_df.loc[[30, 31, 32]]

# Combine results
selected_rows = pd.concat([selected_first, second_category], ignore_index=True)

round(selected_rows, 3)

In [ ]:
selected_indices = [7, 2, 5, 1, 29]

# Extract elements
selected_parameters = [parameters_list[i] for i in selected_indices]

# Print or use the selected elements
print(selected_parameters)

# Information Fairness Tradeoff

In [23]:
# Assuming results_df is already in memory
# Define the fairness and information measures
fairness_measures = ["MMD", "% Delta DP Mean"]
information_measures = ["% Variance", "% Accuracy Mean", 'F1 Mean']

# Define colors and labels
colors = ["#377eb8"] * 30 + ["#e41a1c"] * 3 + ["#984ea3", "#ff7f00", "#4daf4a"]
labels = ["SIRFairRe"] * 30 + ["FairPCA"] * 3 + ["PCA", "SIR(Y1)", "-SIR(Y2)"]

In [ ]:
# Create scatter plots
for fairness in fairness_measures:
    for info in information_measures:
        plt.figure(figsize=(8, 6))
        
        # Plot each point with its assigned color and label
        for i in range(len(results_df)):
            #marker_style = '*' if i in [7, 2, 5, 1, 29, 30, 31, 32] else 'o'
            plt.scatter(results_df[fairness][i], results_df[info][i],
             #           marker=marker_style,
                        color=colors[i], label=labels[i] if i in [0, 30, 33, 34, 35] else "_nolegend_",
                       s=200)

        # Formatting
        plt.xlabel(fairness, fontsize=20)   # Make X label bigger
        plt.ylabel(info, fontsize=20)        # Make Y label bigger
        plt.xticks(fontsize=16)               # Make X ticks bigger
        plt.yticks(fontsize=16)               # Make Y ticks bigger
        
        # Add a horizontal line at accuracy_mean if the Y-axis represents accuracy
        if "Accuracy" in info:
            plt.axhline(y=accuracy_mean * 100, color="red", linestyle="--", linewidth=2, label="Raw Data Accuracy")
        
        #plt.title(f"{info} vs {fairness}")
        plt.legend(fontsize=18)
        plt.grid(True)
        
        # Save the figure
        filename = f"ADULT_scatter_{fairness.replace('% ', '').replace(' ', '_')}_vs_{info.replace('% ', '').replace(' ', '_')}.png"
        plt.savefig(filename, dpi=300, bbox_inches="tight")
        
        plt.show()
        plt.close()

In [ ]:
import matplotlib.lines as mlines

# Define your important points
special_indices = [1, 3, 29, 30, 31, 32]
legend_indices = [0, 30, 33, 34, 35]

# Create scatter plots
for fairness in fairness_measures:
    for info in information_measures:
        plt.figure(figsize=(8, 6))
        
        # Prepare arrays
        x = results_df[fairness].values
        y = results_df[info].values
        colors_to_plot = colors
        sizes_to_plot = [600 if i in special_indices else 200 for i in range(len(results_df))]

        # Plot all points at once
        plt.scatter(x, y,
                    c=colors_to_plot,
                    s=sizes_to_plot,
                    marker='o')
        
        # Formatting
        plt.xlabel(fairness, fontsize=20)
        plt.ylabel(info, fontsize=20)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        
        # Add a horizontal line at accuracy_mean if the Y-axis represents accuracy
        if "Accuracy" in info:
            plt.axhline(y=accuracy_mean * 100, color="red", linestyle="--", linewidth=2, label="Raw Data Accuracy")
        
        # Manually create the legend with uniform marker sizes
        handles = []
        for idx in legend_indices:
            handle = mlines.Line2D([], [],
                                   color=colors[idx],
                                   marker='o',
                                   linestyle='None',
                                   markersize=8,
                                   label=labels[idx])
            handles.append(handle)

        plt.legend(handles=handles, fontsize=18)
        plt.grid(True)
        
        # Save the figure
        filename = f"ADULT_scatter_STARS_{fairness.replace('% ', '').replace(' ', '_')}_vs_{info.replace('% ', '').replace(' ', '_')}.png"
        plt.savefig(filename, dpi=300, bbox_inches="tight")
        
        plt.show()
        plt.close()


# Individual Metrics Visualized with Heatmaps

In [ ]:
# Load data from the file
file_path = "simulation_results.txt"  # Update this if needed
df = pd.read_csv(file_path, sep="\t")  # Read the tab-separated values

# Define the parameter grid
grid_size = (6, 5)  # 8x8 grid
alpha_values = sorted(set(param[0] for param in parameters_list))  # Unique alpha values
lambda_values = sorted(set(param[1] for param in parameters_list))  # Unique lambda values

# Map parameter settings to grid indices
param_to_index = {tuple(parameters_list[i]): i for i in range(len(parameters_list))}

# Define characteristics to plot
characteristics = {
    "MMD": "MMD",
    "% Variance": "% Variance",
    "% Accuracy Mean": "% Accuracy Mean",
    "% Delta DP Mean": "% Delta DP Mean",
    "F1 Mean": "F1 Mean"
}

# Choose a single-color colormap (e.g., "Blues", "Greens", "Reds", "Purples")
cmap_color = "Blues"  

# Create an 8x8 grid for each heatmap
for char_name, col_name in characteristics.items():
    heatmap_data = np.full(grid_size, np.nan)  # Initialize grid with NaNs for missing values
    
    # Populate the heatmap matrix based on parameter mappings
    for (alpha, lambda_), index in param_to_index.items():
        if index in df.index:
            i = alpha_values.index(alpha)
            j = lambda_values.index(lambda_)
            heatmap_data[i, j] = df.at[index, col_name]  # Assign corresponding value
    
    # Set rounding format
    fmt = ".3f" if char_name == "MMD" else ".1f"

    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(heatmap_data, annot=True, fmt=fmt, cmap=cmap_color,
                     xticklabels=lambda_values, yticklabels=alpha_values, annot_kws={"size": 12})

    # Formatting
    plt.xlabel("Lambda", fontsize=20)
    plt.ylabel("Alpha", fontsize=20)
    plt.title(f"Heatmap of {char_name}", fontsize=20)
    plt.xticks(rotation=45, fontsize=16)
    plt.yticks(rotation=0, fontsize=16)

    # Save the heatmap as an image file
    filename = f"ADULT_HEATMAP_{char_name.replace('% ', '').replace(' ', '_')}.png"
    plt.savefig(filename, dpi=300, bbox_inches="tight")

    # Show plot
    plt.show()


In [ ]:
# Load the tab-separated text file into a DataFrame
results_df = pd.read_csv("simulation_results.txt", sep="\t")

# Display the first few rows to verify
print(results_df.head())

In [ ]:
silhouette_scores = []
count = 0
for factor_loading in factor_loadings:
    print(count)
    X_reduced = X @ factor_loading
    score = silhouette_score(X_reduced, Y)
    silhouette_scores.append(score)
    count += 1

In [65]:
# Add Silhouette Scores as a new column in results_df
results_df["Silhouette Score"] = silhouette_scores

In [ ]:
results_df

In [66]:
best_sirfairre = results_df.iloc[:30].loc[results_df.iloc[:30][results_df.iloc[:30]["MMD"] <= 0.1]["Silhouette Score"].idxmax()]
best_fairpca = results_df.iloc[30:33].loc[results_df.iloc[30:33][results_df.iloc[30:33]["MMD"] <= 0.1]["Silhouette Score"].idxmax()]

In [ ]:
best_sirfairre

In [ ]:
best_fairpca

# Reduced Dim plots

In [ ]:
plot_reduced_X(X, factor_loadings[3], Z, "SIRFairRe", "Gender", "Female", "Male", caption="MMD = 0.076; ΔDP = 4.8%")
plot_reduced_X(X, factor_loadings[3], Y, "SIRFairRe", "Income", "<= 50K", "> 50K", caption="Var = 5.5%; Acc = 75.2%; SS = 0.057")

In [ ]:
plot_reduced_X(X, factor_loadings[31], Z, "FairPCA",  "Gender", "Female", "Male", caption="MMD = 0.017; ΔDP = 0%")
plot_reduced_X(X, factor_loadings[31], Y, "FairPCA", "Income", "<= 50K", "> 50K", caption="Var = 4.4%; Acc = 75.2%; SS = 0.0050")

In [ ]:
# Define indices for PCA, SIR(Y1), and -SIR(Y2)
factor_indices = {"PCA": 33, "SIR(Y1)": 34, "-SIR(Y2)": 35}

# Define target variable settings with standardized caption names
targets = [
    ("Gender", "Female", "Male", "MMD", "% Delta DP Mean"),  # MMD and ΔDP for Gender
    ("Income", "<= 50K", "> 50K", "% Variance", "% Accuracy Mean", "% Delta DP Mean")  # Variance, Accuracy, and ΔDP for Income
]

# Define standardized caption name replacements
caption_renames = {
    "MMD": "MMD",
    "% Variance": "Var",
    "% Accuracy Mean": "Acc",
    "% Delta DP Mean": "ΔDP"
}

# Generate plots
for method_name, idx in factor_indices.items():
    for target, group1, group2, *metric_keys in targets:
        
        # Extract corresponding performance metrics
        metrics = results_df.loc[results_df["Factor_Loading_Index"] == idx, metric_keys].values.flatten()

        # Format caption dynamically with different rounding for each metric
        formatted_metrics = []
        for i, key in enumerate(metric_keys):
            new_name = caption_renames[key]

            if new_name == "MMD":  # Round to 3 decimal places
                formatted_metrics.append(f"{new_name} = {metrics[i]:.3f}")
            elif new_name == "Var":  # Round to 2 decimal places and add '%'
                formatted_metrics.append(f"{new_name} = {metrics[i]:.2f}%")
            elif new_name in ["Acc", "ΔDP"]:  # Round to 1 decimal place and add '%'
                formatted_metrics.append(f"{new_name} = {metrics[i]:.1f}%")
            else:  # Default case (shouldn't happen)
                formatted_metrics.append(f"{new_name} = {metrics[i]}")

        caption = "; ".join(formatted_metrics)

        # Generate plot
        plot_reduced_X(X, factor_loadings[idx], Z if target == "Gender" else Y, method_name, target, group1, group2, caption=caption)


In [ ]:
np.unique(Z)

In [ ]:
np.unique(Y)

# Redo all with d = 10

# Compute All Loadings of Interest

## Option 1

In [ ]:
Vs10 = dict()

In [ ]:
for setting in parameters_list:
    al = tuple(setting)
    a, l, = al
    V = method1(X, Y, Z, 10, a, l)
    Vs10[al] = V

In [ ]:
len(Vs)

## PCA

In [ ]:
Sigma = np.cov(X, rowvar=False)

In [ ]:
eigenvalues, eigenvectors = eigh(Sigma)

In [ ]:
V_PCA = eigenvectors[:, -10:]

## SIR(Y1)

In [ ]:
V_SIRY1 = SIR(X, Y, 10)

## -SIR(Y2)

In [ ]:
V_negSIRY2 = method1(X, Y, Z, 10, 0, 0)

## FairPCA

In [ ]:
Vs_fair = dict()

In [ ]:
ms = [1, 15, 50]

In [ ]:
for m in ms:
    V = fairPCA(X, Z, 10, m)
    Vs_fair[m] = V

In [ ]:
len(ms)

# Perform Data Splits

In [ ]:
factor_loadings10 = list(Vs10.values()) + list(Vs_fair.values())
factor_loadings10.append(V_PCA)
factor_loadings10.append(V_SIRY1)
factor_loadings10.append(V_negSIRY2)

In [ ]:
sample_frac = 0.1
C = 1
gamma = "scale"
kernel = "linear"
cv = 5
splits = get_splits(X, Y, cv=cv)

In [ ]:
from tqdm import tqdm
from joblib import Parallel, delayed

# Wrap `enumerate(factor_loadings)` with `list()` so `tqdm` can track progress
results = Parallel(n_jobs=-1, verbose=10)(
    delayed(process_factor_loading)(V, i, X, Y, Z, sample_frac, kernel, C, gamma, cv, splits) 
    for i, V in tqdm(list(enumerate(factor_loadings)), total=len(factor_loadings))
)

# Convert list of results to a DataFrame
results_df = pd.DataFrame(results)

# Save as a tab-separated text file
results_df.to_csv("simulation_results_d=10.txt", sep="\t", index=False)

In [39]:
# Load the results from the saved file
results_df = pd.read_csv("simulation_results_d=10.txt", sep="\t")

# Information Fairness Tradeoff

In [40]:
# Assuming results_df is already in memory
# Define the fairness and information measures
fairness_measures = ["MMD", "% Delta DP Mean"]
information_measures = ["% Variance", "% Accuracy Mean"]

# Define colors and labels
colors = ["blue"] * 30 + ["red"] * 3 + ["yellow", "orange", "green"]
labels = ["SIRFairRe"] * 30 + ["FairPCA"] * 3 + ["PCA", "SIR(Y1)", "-SIR(Y2)"]

In [ ]:
# Create scatter plots
for fairness in fairness_measures:
    for info in information_measures:
        plt.figure(figsize=(8, 6))
        
        # Plot each point with its assigned color and label
        for i in range(len(results_df)):
            plt.scatter(results_df[fairness][i], results_df[info][i],
                        color=colors[i], label=labels[i] if i in [0, 30, 33, 34, 35] else "_nolegend_")

        # Formatting
        plt.xlabel(fairness, fontsize=20)   # Make X label bigger
        plt.ylabel(info, fontsize=20)        # Make Y label bigger
        plt.xticks(fontsize=16)               # Make X ticks bigger
        plt.yticks(fontsize=16)               # Make Y ticks bigger
        
        # Add a horizontal line at accuracy_mean if the Y-axis represents accuracy
        if "Accuracy" in info:
            plt.axhline(y=accuracy_mean * 100, color="red", linestyle="--", linewidth=2, label="Raw Data Accuracy")
        
        #plt.title(f"{info} vs {fairness}")
        plt.legend(fontsize=18)
        plt.grid(True)
        
        # Save the figure
        filename = f"ADULT_scatter_d=10_{fairness.replace('% ', '').replace(' ', '_')}_vs_{info.replace('% ', '').replace(' ', '_')}.png"
        plt.savefig(filename, dpi=300, bbox_inches="tight")
        
        plt.show()
        plt.close()

# Individual Metrics Visualized with Heatmaps

In [ ]:
# Load data from the file
file_path = "simulation_results_d=10.txt"  # Update this if needed
df = pd.read_csv(file_path, sep="\t")  # Read the tab-separated values

# Define the parameter grid
grid_size = (6, 5)  # 8x8 grid
alpha_values = sorted(set(param[0] for param in parameters_list))  # Unique alpha values
lambda_values = sorted(set(param[1] for param in parameters_list))  # Unique lambda values

# Map parameter settings to grid indices
param_to_index = {tuple(parameters_list[i]): i for i in range(len(parameters_list))}

# Define characteristics to plot
characteristics = {
    "MMD": "MMD",
    "% Variance": "% Variance",
    "% Accuracy Mean": "% Accuracy Mean",
    "% Delta DP Mean": "% Delta DP Mean",
     "F1 Mean": "F1 Mean"
}

# Choose a single-color colormap (e.g., "Blues", "Greens", "Reds", "Purples")
cmap_color = "Blues"  

# Create an 8x8 grid for each heatmap
for char_name, col_name in characteristics.items():
    heatmap_data = np.full(grid_size, np.nan)  # Initialize grid with NaNs for missing values
    
    # Populate the heatmap matrix based on parameter mappings
    for (alpha, lambda_), index in param_to_index.items():
        if index in df.index:
            i = alpha_values.index(alpha)
            j = lambda_values.index(lambda_)
            heatmap_data[i, j] = df.at[index, col_name]  # Assign corresponding value
    
    # Set rounding format
    fmt = ".3f" if char_name == "MMD" else ".1f"

    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(heatmap_data, annot=True, fmt=fmt, cmap=cmap_color,
                     xticklabels=lambda_values, yticklabels=alpha_values)

    # Formatting
    plt.xlabel("Lambda")
    plt.ylabel("Alpha")
    plt.title(f"Heatmap of {char_name}")
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)

    # Save the heatmap as an image file
    filename = f"ADULT_HEATMAP_d=10_{char_name.replace('% ', '').replace(' ', '_')}.png"
    plt.savefig(filename, dpi=300, bbox_inches="tight")

    # Show plot
    plt.show()


In [ ]:
# Load the tab-separated text file into a DataFrame
results_df = pd.read_csv("simulation_results_d=10.txt", sep="\t")

# Display the first few rows to verify
print(results_df.head())

In [ ]:
silhouette_scores = []
count = 0
for factor_loading in factor_loadings:
    print(count)
    X_reduced = X @ factor_loading
    score = silhouette_score(X_reduced, Y)
    silhouette_scores.append(score)
    count += 1

In [ ]:
# Add Silhouette Scores as a new column in results_df
results_df["Silhouette Score"] = silhouette_scores

In [ ]:
best_sirfairre = results_df.iloc[:30].loc[results_df.iloc[:30][results_df.iloc[:30]["MMD"] <= 0.1]["Silhouette Score"].idxmax()]
best_fairpca = results_df.iloc[30:33].loc[results_df.iloc[30:33][results_df.iloc[30:33]["MMD"] <= 0.1]["Silhouette Score"].idxmax()]

In [ ]:
best_sirfairre

In [ ]:
best_fairpca